In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess dataset
df = pd.read_csv('network_data.csv')  # Replace with actual dataset path
X = df.drop('label', axis=1)
y = df['label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Function to evaluate models
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy: {accuracy}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))
    print("Classification Report:\n", classification_report(y_test, predictions))

# 1. Train and evaluate using RandomForest with GridSearchCV for hyperparameter tuning
rf_model = RandomForestClassifier()
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
grid_rf = GridSearchCV(rf_model, param_grid_rf, cv=3)
grid_rf.fit(X_train, y_train)
evaluate_model(grid_rf.best_estimator_, X_test, y_test)

# 2. Train and evaluate using XGBoost
xgb_model = XGBClassifier()
param_grid_xgb = {'max_depth': [3, 6, 10], 'n_estimators': [50, 100, 200]}
grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3)
grid_xgb.fit(X_train, y_train)
evaluate_model(grid_xgb.best_estimator_, X_test, y_test)

# 3. Deep learning model for IDS
dl_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
dl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dl_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)
dl_preds = dl_model.predict(X_test)
dl_preds = (dl_preds > 0.5).astype(int).reshape(-1)
print("Deep Learning Model Evaluation:")
evaluate_model(dl_model, X_test, y_test)